In [1]:
import numpy as np
import mne
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import RMSprop


import numpy as np
import mne
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings("ignore")

2025-03-10 09:31:30.942695: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-10 09:31:30.959403: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-10 09:31:30.964424: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 09:31:30.979381: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-10 09:31:31.964721: W tensorflow/compiler/tf2

In [2]:


def process_physionet_data(subject_ids, channels, T=160, delta=4, Delta=8, Gamma=30):
    X_train, y_train = [], []

    for i, subject in enumerate(subject_ids):

        raw = mne.io.read_raw_edf(f'./files/S{subject:03d}/S{subject:03d}R01.edf', preload=True)
        raw.pick_channels(channels)

        data = raw.get_data().T
        scaler = MinMaxScaler()
        normalized_data = scaler.fit_transform(data)

        n_samples = normalized_data.shape[0]
        windows = [normalized_data[start:start+T, :] for start in range(0, n_samples - T + 1, delta)]
        windows = np.array(windows)

        augmented = [windows[i:i+Gamma] for i in range(0, len(windows) - Gamma + 1, Delta)]
        augmented = np.array(augmented)

        X_train.append(augmented)
        y_train.extend([i] * len(augmented))
        
    X_train = np.concatenate(X_train, axis=0)
    y_train = np.array(y_train)

    return X_train, y_train

channels = ['Oz..', 'T7..', 'Cz..']
alpha_subjects = list(range(1, 91))
beta_subjects = list(range(91, 110))
X_train, y_train = process_physionet_data(alpha_subjects, channels)
X_test, y_test = process_physionet_data(beta_subjects, channels)


Extracting EDF parameters from /home/jovyan/Authenticators/files/S001/S001R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Extracting EDF parameters from /home/jovyan/Authenticators/files/S002/S002R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Extracting EDF parameters from /home/jovyan/Authenticators/files/S003/S003R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Extracting EDF parameters from /home/jovyan/Authenticators/files/S004/S004R01.edf...
EDF fi

In [3]:
def build_classification_model(input_shape, n_classes):
    inputs = Input(shape=input_shape)
    x = Conv2D(128, (3,3), activation='relu')(inputs)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(256, (3,3), activation='relu')(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(512, (3,3), activation='relu')(x)
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.25)(x)
    outputs = Dense(n_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer=RMSprop(0.0001), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
    return model


model = build_classification_model((30, 160, 3), len(alpha_subjects))
model.fit(X_train, y_train, epochs=30, batch_size=64)


fingerprint_model = Model(inputs=model.input, outputs=model.layers[-4].output)

2025-03-10 09:31:37.235278: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79077 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:01:00.0, compute capability: 8.0


Epoch 1/30


I0000 00:00:1741599102.859479 1298784 service.cc:146] XLA service 0x7fb8e4005830 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741599102.859536 1298784 service.cc:154]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2025-03-10 09:31:42.880348: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-03-10 09:31:43.002229: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902
2025-03-10 09:31:44.628175: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_641', 100 bytes spill stores, 100 bytes spill loads



 23/418 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.0078 - loss: 4.5070

I0000 00:00:1741599107.947208 1298784 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


412/418 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0862 - loss: 3.9206

2025-03-10 09:31:51.027261: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_641', 12 bytes spill stores, 12 bytes spill loads



418/418 ━━━━━━━━━━━━━━━━━━━━ 12s 14ms/step - accuracy: 0.0873 - loss: 3.9128
Epoch 2/30
418/418 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.2949 - loss: 2.6230
Epoch 3/30
418/418 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.4066 - loss: 2.0839
Epoch 4/30
418/418 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.4770 - loss: 1.8107
Epoch 5/30
418/418 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5362 - loss: 1.5617
Epoch 6/30
418/418 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5912 - loss: 1.3575
Epoch 7/30
418/418 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6394 - loss: 1.1735
Epoch 8/30
418/418 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6817 - loss: 1.0264
Epoch 9/30
418/418 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.7124 - loss: 0.8981
Epoch 10/30
418/418 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.7435 - loss: 0.8093
Epoch 11/30
418/418 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.7775 - loss: 0.6974
Epoch 12/30
418/418 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accura

In [4]:
from sklearn.metrics import pairwise_distances

def enroll_user(eeg_sample):
    fingerprint = fingerprint_model.predict(eeg_sample, verbose=0)
    return fingerprint.mean(axis=0) 

def authenticate(claimed_fingerprint, new_sample, threshold=0.275):
    new_fingerprint = fingerprint_model.predict(new_sample, verbose=0).mean(axis=0)
    distance = pairwise_distances([claimed_fingerprint], [new_fingerprint], metric='cosine')[0][0]
    return distance <= threshold


In [5]:
from sklearn.model_selection import train_test_split

test_size = 0.1  # 10%
X_train_sampled, _, y_train_sampled, _ = train_test_split(X_train, y_train, test_size=1-test_size, stratify=y_train, random_state=42)
X_test_sampled, _, y_test_sampled, _ = train_test_split(X_test, y_test, test_size=1-test_size, stratify=y_test, random_state=42)

In [6]:
X_train_sampled.shape

(2671, 30, 160, 3)

In [7]:
X_test_sampled.shape

(563, 30, 160, 3)

In [8]:
import numpy as np
from sklearn.metrics import pairwise_distances, accuracy_score

def batch_test_authentication(X_test, y_test, threshold=0.275):
    
    user_fingerprints = {}
    predictions = []
    labels = []
    
    unique_users = np.unique(y_test)
    
    for user in unique_users:
        user_samples = X_test[y_test == user]
        user_fingerprints[user] = enroll_user(user_samples)
    
    for i in range(len(X_test)):
        true_user = y_test[i]
        new_sample = X_test[i:i+1]
        
        claimed_fingerprint = user_fingerprints[true_user]
        is_authenticated = authenticate(claimed_fingerprint, new_sample, threshold)
        
        predictions.append(int(is_authenticated))
        labels.append(1)

        other_user = np.random.choice(unique_users[unique_users != true_user])
        claimed_fingerprint = user_fingerprints[other_user]
        is_authenticated = authenticate(claimed_fingerprint, new_sample, threshold)
        
        predictions.append(int(is_authenticated))
        labels.append(0)

    accuracy = accuracy_score(labels, predictions)
    return accuracy




In [9]:
Train_accuracy = batch_test_authentication(X_train_sampled, y_train_sampled)
print(f"Authentication Accuracy: {Train_accuracy * 100:.2f}%")

E0000 00:00:1741599194.685557 1298783 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1741599194.859801 1298783 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1741599195.534273 1298782 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1741599195.708358 1298782 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


Authentication Accuracy: 88.82%


In [10]:
Test_accuracy = batch_test_authentication(X_test_sampled, y_test_sampled)
print(f"Authentication Accuracy: {Test_accuracy * 100:.2f}%")

Authentication Accuracy: 84.37%
